## Chapter 2-6, 2강 NLP 임베딩 기반 검색 — TF‑IDF vs SBERT vs Hybrid (CPU)

- 목표: 의미/키워드 검색 방식을 비교하고, 하이브리드로 직관적 성능 차이를 체감
- 제약: CPU 전용 환경(훈련/추론 모두 CPU)
- 데이터: AG News 코퍼스(소규모 서브셋)


### 구성 (Overview)
- 데이터 로드 및 corpus 생성 → 임베딩 (SBERT, TF‑IDF) → 질문 검색 (TF‑IDF / SBERT / Hybrid) → 결과 비교: 직관적 성능 분석(토픽/키워드/문맥)


### 0. 환경 설정 및 라이브러리


In [118]:
# =========================
# 0. 환경 설정 및 라이브러리
# =========================

# 표준 라이브러리
import os, time, random
from typing import List, Tuple
from contextlib import contextmanager

# 수치/데이터/시각화
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 사이킷런
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction import text as sk_text

# datasets / sentence-transformers (CPU 고정)
from datasets import load_dataset
from sentence_transformers import SentenceTransformer

import warnings
warnings.filterwarnings("ignore", message=".*matmul.*")

# -----------------------------------------
# Matplotlib: 한글 폰트 및 마이너스 기호 설정
# -----------------------------------------
plt.rcParams["font.family"] = "AppleGothic"
plt.rcParams["axes.unicode_minus"] = False


In [119]:
# ------------------------
# 재현성(시드) 고정
# ------------------------
def set_seed(seed: int = 42) -> None:
    random.seed(seed)
    np.random.seed(seed)

set_seed(42)

# ------------------------
# 간단 타이머
# ------------------------
@contextmanager
def timer(msg: str):
    t0 = time.perf_counter()
    yield
    print(f"[TIME] {msg}: {time.perf_counter() - t0:.2f}s")

### 1. 데이터 로드 및 corpus 생성 (AG News)


In [120]:
# ------------------------
# AG News 서브셋 로더
# ------------------------
label_names: List[str] = ["World", "Sports", "Business", "Sci/Tech"]

def load_ag_news_subset(n_per_class: int = 200) -> Tuple[List[str], List[int]]:
    """
    AG News에서 클래스별로 동일 개수(n_per_class)만큼 샘플을 추출해
    텍스트 리스트(xs)와 정수 라벨 리스트(ys)를 반환합니다.

    Args:
        n_per_class (int): 각 클래스(0~3)에서 가져올 샘플 수. 기본 200.

    Returns:
        xs (List[str]): 뉴스 텍스트 리스트
        ys (List[int]): 라벨 리스트 (0=World, 1=Sports, 2=Business, 3=Sci/Tech)
    """
    # -------------------------
    # 1) AG News train split 로드 (총 120k 샘플)
    # -------------------------
    ds = load_dataset("ag_news", split="train")

    xs, ys = [], []

    # -------------------------
    # 2) 클래스별 균등 샘플링
    #    - 라벨(0~3) 기준으로 필터링
    #    - 앞에서 n_per_class개만 선택
    # -------------------------
    for lab in range(4):
        sub = ds.filter(lambda ex: ex["label"] == lab)
        sub = sub.select(range(n_per_class))

        xs += [r["text"] for r in sub]
        ys += [int(r["label"]) for r in sub]
        
    return xs, ys

In [121]:
with timer("AG News 로드"):
    # 각 클래스에서 200개씩 불러와 텍스트/라벨 리스트 생성
    # label_names: ["World", "Sports", "Business", "Sci/Tech"]
    corpus_texts, corpus_labels = load_ag_news_subset(n_per_class=200)

# 전체 샘플 수와 라벨 이름 확인
print(len(corpus_texts), "samples")
print("label_names:", label_names)

# -------------------------
# 클래스별 샘플 예시 2개씩 출력
# - shown 딕셔너리로 클래스별 출력 개수 제한(2개)
# - 텍스트는 길수 있으므로 앞 120자만 미리보기 형태로 출력
# -------------------------
print("\n[클래스별 샘플 예시]")
shown = {i: 0 for i in range(4)}  # 각 라벨별로 몇 개를 보여줬는지 카운트
for t, lab in zip(corpus_texts, corpus_labels):
    if shown[lab] < 2:
        print(f"[{label_names[lab]}] {str(t)[:120]}...")
        shown[lab] += 1



[TIME] AG News 로드: 2.07s
800 samples
label_names: ['World', 'Sports', 'Business', 'Sci/Tech']

[클래스별 샘플 예시]
[World] Venezuelans Vote Early in Referendum on Chavez Rule (Reuters) Reuters - Venezuelans turned out early\and in large number...
[World] S.Koreans Clash with Police on Iraq Troop Dispatch (Reuters) Reuters - South Korean police used water cannon in\central ...
[Sports] Phelps, Thorpe Advance in 200 Freestyle (AP) AP - Michael Phelps took care of qualifying for the Olympic 200-meter frees...
[Sports] Reds Knock Padres Out of Wild-Card Lead (AP) AP - Wily Mo Pena homered twice and drove in four runs, helping the Cincinn...
[Business] Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics,...
[Business] Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputat...
[Sci/Tech] 'Madden,' 'ESPN' Football Score in Different Ways (Reuters) Reuters - Was absente

### 2. 임베딩 2가지: SBERT / TF‑IDF


In [122]:
# ------------------------
# TF-IDF 인덱스 구축
# ------------------------
# stopwords: 영어 불용어 사용(키워드 매칭 과다 방지)
stop_words = list(sk_text.ENGLISH_STOP_WORDS) 

In [123]:
stop_words

['every',
 'therefore',
 'until',
 'will',
 'my',
 'from',
 'only',
 'thick',
 'everywhere',
 'well',
 'who',
 'detail',
 'hereby',
 'name',
 'whenever',
 'nowhere',
 'or',
 'around',
 'co',
 'everything',
 'becoming',
 'behind',
 'three',
 'top',
 'after',
 'front',
 'at',
 'down',
 'anyhow',
 'once',
 'see',
 'done',
 'through',
 'formerly',
 'us',
 'wherein',
 'put',
 'i',
 'due',
 'below',
 'whether',
 'are',
 'hundred',
 'elsewhere',
 'yet',
 'forty',
 'perhaps',
 'again',
 'upon',
 'made',
 'ltd',
 'towards',
 'to',
 'etc',
 'now',
 'however',
 'meanwhile',
 'also',
 'already',
 'least',
 'cannot',
 're',
 'become',
 'each',
 'among',
 'latterly',
 'because',
 'is',
 'few',
 'beforehand',
 'have',
 'bottom',
 'amount',
 'am',
 'not',
 'never',
 'move',
 'as',
 'than',
 'whither',
 'herself',
 'do',
 'up',
 'whom',
 'along',
 'ten',
 'describe',
 'could',
 'hers',
 'ourselves',
 'therein',
 'thin',
 'no',
 'against',
 'ours',
 'her',
 'else',
 'eight',
 'take',
 'found',
 'anywher

In [124]:
# -------------------------
# TF-IDF 벡터화
# -------------------------
# - max_features=30000 : 전체 단어/바이그램 중 상위 3만 개만 사용(빈도 기준)
# - ngram_range=(1, 2) : 유니그램과 바이그램을 함께 사용
# - stop_words        : 불용어 리스트(예: 영어면 'english' 또는 커스텀 리스트)
vectorizer = TfidfVectorizer(
    max_features=30000,
    ngram_range=(1, 2),
    stop_words=stop_words
)

with timer("TF-IDF fit_transform(corpus)"):
    X_tfidf = vectorizer.fit_transform(corpus_texts)

# 결과 차원: (문서 개수, 사용된 토큰 개수)
print("TF-IDF shape:", X_tfidf.shape)

[TIME] TF-IDF fit_transform(corpus): 0.09s
TF-IDF shape: (800, 21987)


In [125]:
# ------------------------
# SBERT 임베딩 구축 (CPU)
# ------------------------

def sbert_encode(texts: List[str], batch_size: int = 64,
                 model_name: str = "sentence-transformers/all-MiniLM-L6-v2") -> np.ndarray:
    """
    Sentence-BERT 모델로 문장 임베딩을 생성합니다. (CPU 기본)
    - encode()의 normalize_embeddings=False로 두고, 아래에서 수동 L2 정규화 수행
    - NaN/Inf 방지도 함께 처리

    Args:
        texts (List[str]): 인코딩할 문장(문서) 리스트
        batch_size (int): 배치 크기 (메모리/속도 트레이드오프)
        model_name (str): sentence-transformers 허브 모델 이름

    Returns:
        np.ndarray: (N, D) 형태의 L2-정규화된 임베딩 배열 (float64)
    """
    # 모델 로드 (CPU). GPU 사용 시 device="cuda"로 변경 가능
    model = SentenceTransformer(model_name, device="cpu")

    arrs = []
    # 배치 단위로 순회
    for i in range(0, len(texts), batch_size):
        # convert_to_numpy=True: 바로 NumPy 배열 반환
        # normalize_embeddings=False: 아래에서 직접 L2 정규화 예정
        arr = model.encode(
            texts[i:i+batch_size],
            batch_size=batch_size,
            show_progress_bar=False,
            convert_to_numpy=True,
            normalize_embeddings=False
        ).astype(np.float64)

        # ----- 안정적인 L2 정규화 -----
        # 분모가 0이 되는 경우를 방지하기 위해 clip으로 하한선 설정
        norms = np.linalg.norm(arr, axis=1, keepdims=True)
        arr = arr / np.clip(norms, 1e-12, None)

        # 숫자 안정성: NaN/±Inf → 0.0 치환
        arr = np.nan_to_num(arr, nan=0.0, posinf=0.0, neginf=0.0)

        arrs.append(arr)

    # 배치 결과를 하나의 (N, D) 배열로 결합
    return np.vstack(arrs)



In [126]:
# -------------------------
# 실행: 코퍼스 임베딩 생성
# -------------------------
with timer("SBERT encode(corpus)"):
    # sbert_encode:
    # - Sentence-BERT로 코퍼스의 각 문장을 D차원 임베딩으로 변환
    X_sbert = sbert_encode(corpus_texts)

# 임베딩 배열의 크기 출력 (예: (800, 384) → 800문서, 384차원 임베딩)
print("SBERT shape:", X_sbert.shape)

[TIME] SBERT encode(corpus): 22.85s
SBERT shape: (800, 384)


### 3. 단일 질문 검색: TF‑IDF / SBERT / Hybrid


In [127]:
# ------------------------
# 검색 함수 3종
# ------------------------

def search_tfidf(query: str, topk: int = 5):
    """
    TF-IDF 기반 키워드 매칭 검색.
    - vectorizer: 이미 fit된 TfidfVectorizer
    - X_tfidf  : (N, V) CSR 희소행렬. 일반적으로 L2 정규화(norm='l2')가 기본.
    - 코사인 유사도 기반으로 가장 유사한 문서 인덱스와 점수를 반환.

    Args:
        query (str): 질의문
        topk (int): 상위 반환 개수

    Returns:
        List[Tuple[int, float]]: (문서 인덱스, 유사도 점수) 리스트
    """
    # (1, V) sparse row
    qv = vectorizer.transform([query])
    # sparse 간 연산 지원. vectorizer가 L2 정규화했다면 linear_kernel로 대체 가능.
    sims = cosine_similarity(qv, X_tfidf)[0]  # shape: (N,)
    idx = np.argsort(-sims)[:topk]
    
    return [(int(i), float(sims[i])) for i in idx]

In [128]:
def search_sbert(query: str, topk: int = 5):
    """
    SBERT(문장 임베딩) 기반 [유사의미] 검색.
    - sbert_encode: 내부에서 L2 정규화 수행 → 코사인 유사도 = 내적
    - X_sbert     : (N, D) 실수 배열, 각 행은 단위벡터.

    Args:
        query (str): 질의문
        topk (int): 상위 반환 개수

    Returns:
        List[Tuple[int, float]]: (문서 인덱스, 유사도 점수) 리스트
    """
    # (1, D) 배열
    qv = sbert_encode([query])           # 이미 L2 정규화된 단위벡터
    sims = (X_sbert @ qv.T).ravel()      # 내적 = 코사인 유사도
    idx = np.argsort(-sims)[:topk]
    
    return [(int(i), float(sims[i])) for i in idx]

In [129]:
def search_hybrid(query: str, topk: int = 5, alpha: float = 0.5):
    """
    하이브리드 검색(TF-IDF + SBERT 가중 합).
    - alpha=1.0이면 SBERT만, 0.0이면 TF-IDF만 사용.
    - 서로 다른 점수 스케일을 단순 합산하므로, alpha는 데이터/도메인에 맞게 튜닝 권장.

    Args:
        query (str): 질의문
        topk (int): 상위 반환 개수
        alpha (float): SBERT 가중치 [0, 1]

    Returns:
        List[Tuple[int, float]]: (문서 인덱스, 하이브리드 점수) 리스트
    """
    # TF-IDF 쿼리 벡터 및 유사도
    qv_tfidf = vectorizer.transform([query])
    sims_tfidf = cosine_similarity(qv_tfidf, X_tfidf)[0]

    # SBERT 쿼리 벡터 및 유사도(내적=코사인)
    qv_sbert = sbert_encode([query])
    sims_sbert = (X_sbert @ qv_sbert.T).ravel()

    # 가중 합
    sims = alpha * sims_sbert + (1 - alpha) * sims_tfidf
    idx = np.argsort(-sims)[:topk]
    
    return [(int(i), float(sims[i])) for i in idx]


In [130]:
query = "What is the impact of interest rate hikes on the stock market?"

In [131]:
# ------------------------
# 질의에 대한 검색 및 결과 출력
# ------------------------
#

# 결과 수집 리스트
rows_tfidf, rows_sbert, rows_hybrid = [], [], []

# ------------------------
# TF-IDF 검색 및 결과 수집
# ------------------------
# search_tfidf(query, topk=5) → [(문서인덱스, 유사도점수), ...]
for k, (i, s) in enumerate(search_tfidf(query, topk=5), start=1):
    print(f"- ({s:.3f}) [{label_names[corpus_labels[i]]}] {corpus_texts[i]}")
    rows_tfidf.append({
        "rank": k,
        "score": float(s),
        "doc_id": int(i),
        "label": label_names[corpus_labels[i]],
        "text": corpus_texts[i]
    })

- (0.188) [Business] Veteran inventor in market float Trevor Baylis, the veteran inventor famous for creating the Freeplay clockwork radio, is planning to float his company on the stock market.
- (0.180) [Business] In a Down Market, Head Toward Value Funds There is little cause for celebration in the stock market these days, but investors in value-focused mutual funds have reason to feel a bit smug -- if only because they've lost less than the folks who stuck with growth.
- (0.172) [Business] Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
- (0.172) [Business] Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market this week during the depth of the\summer doldrums.
- (0.170) [Busi

In [132]:
# ------------------------
# SBERT 검색 및 결과 수집
# ------------------------
# search_sbert(query, topk=5) → [(문서인덱스, 코사인유사도), ...]
for k, (i, s) in enumerate(search_sbert(query, topk=5), start=1):
    print(f"- ({s:.3f}) [{label_names[corpus_labels[i]]}] {corpus_texts[i]}")
    rows_sbert.append({
        "rank": k,
        "score": float(s),
        "doc_id": int(i),
        "label": label_names[corpus_labels[i]],
        "text": corpus_texts[i]
    })

- (0.458) [World] Election-Year Rate Hike Puzzles Some WASHINGTON - Going against conventional wisdom, the Federal Reserve is raising interest rates in an election year. And it is Fed Chairman Alan Greenspan, a Republican, who is leading the charge even though an incumbent Republican in the White House is facing voter unrest about the state of the economy...
- (0.434) [Business] South Korea lowers interest rates South Korea's central bank cuts interest rates by a quarter percentage point to 3.5 in a bid to drive growth in the economy.
- (0.396) [Business] Stocks Higher on Oil Price Relief (Reuters) Reuters - U.S. stocks gained on Monday, getting\a boost from lower oil prices after news the Venezuelan\president survived a recall eased fears about the country's oil\exports.
- (0.374) [Business] Will Schwab Reward Patience? The company saw an improvement in its trades, but will this market be kind to the brokerages?
- (0.373) [World] Stocks Higher Despite Soaring Oil Prices NEW YORK - Wal

In [133]:
# ------------------------
# Hybrid 검색 및 결과 수집
# ------------------------
# search_hybrid(query, topk=5, alpha=0.6)
# - alpha: SBERT 가중치(0~1). 1.0=SBERT만, 0.0=TF-IDF만
for k, (i, s) in enumerate(search_hybrid(query, topk=5, alpha=0.6), start=1):
    print(f"- ({s:.3f}) [{label_names[corpus_labels[i]]}] {corpus_texts[i]}")
    rows_hybrid.append({
        "rank": k,
        "score": float(s),
        "doc_id": int(i),
        "label": label_names[corpus_labels[i]],
        "text": corpus_texts[i]
    })

- (0.298) [World] Election-Year Rate Hike Puzzles Some WASHINGTON - Going against conventional wisdom, the Federal Reserve is raising interest rates in an election year. And it is Fed Chairman Alan Greenspan, a Republican, who is leading the charge even though an incumbent Republican in the White House is facing voter unrest about the state of the economy...
- (0.282) [Business] Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
- (0.279) [Business] Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market this week during the depth of the\summer doldrums.
- (0.270) [Business] Oil and Economy Cloud Stocks' Outlook  NEW YORK (Reuters) - Soaring crude prices plus worries  about the econo

In [134]:
query = "what kind of techninal skill is important to get a job?"

In [135]:
# 결과 수집 리스트
rows_tfidf, rows_sbert, rows_hybrid = [], [], []

In [136]:
for k, (i, s) in enumerate(search_tfidf(query, topk=5), start=1):
    print(f"- ({s:.3f}) [{label_names[corpus_labels[i]]}] {corpus_texts[i]}")
    rows_tfidf.append({
        "rank": k,
        "score": float(s),
        "doc_id": int(i),
        "label": label_names[corpus_labels[i]],
        "text": corpus_texts[i]
    })

- (0.117) [Business] Will Schwab Reward Patience? The company saw an improvement in its trades, but will this market be kind to the brokerages?
- (0.116) [Business] Krispy Kreme COO Leaves for New Job  CHICAGO (Reuters) - Krispy Kreme Doughnuts Inc.  &lt;A HREF="http://www.investor.reuters.com/FullQuote.aspx?ticker=KKD.N target=/stocks/quickinfo/fullquote"&gt;KKD.N&lt;/A&gt;  said on Monday that its chief operating officer is leaving the  company for another job, the latest setback for a company that  is trying to recover from sluggish sales and an accounting  probe.
- (0.086) [Sports] DiMarco, Riley Get on Ryder Cup Team (AP) AP - Hal Sutton had a good idea what kind of U.S. team he would take to the Ryder Cup. All that changed in the final round of the PGA Championship.
- (0.085) [World] President's fate is on the line in Venezuela CARACAS -- Partisans on both sides are calling it the most polarized and important election in Venezuela's history, a presidential recall referendum today

In [137]:
for k, (i, s) in enumerate(search_sbert(query, topk=5), start=1):
    print(f"- ({s:.3f}) [{label_names[corpus_labels[i]]}] {corpus_texts[i]}")
    rows_sbert.append({
        "rank": k,
        "score": float(s),
        "doc_id": int(i),
        "label": label_names[corpus_labels[i]],
        "text": corpus_texts[i]
    })

- (0.316) [Business] Indians fill rail skills shortage Network Rail flies in specialist Indian engineers to work on the West Coast Mainline because of a UK skills shortage.
- (0.252) [World] Device 'may aid kidney dialysis' Scientists are developing a device which could improve the lives of kidney dialysis patients.
- (0.232) [Sci/Tech] New NASA Supercomputer to Aid Theorists and Shuttle Engineers (SPACE.com) SPACE.com - NASA researchers have teamed up with a pair of Silicon Valley firms to build \  a supercomputer that ranks alongside the world's largest Linux-based systems.
- (0.231) [Business] Delightful Dell The company's results show that it's not grim all over tech world. Just all of it that isn't Dell.
- (0.225) [Business] Microsoft Corp. 2.0: a kinder corporate culture Even a genius can mess up. Bill Gates was a brilliant technologist when he cofounded Microsoft , but as he guided it to greatness in both size and historical consequence, he blundered. He terrorized underlings wi

In [138]:
for k, (i, s) in enumerate(search_hybrid(query, topk=5), start=1):
    print(f"- ({s:.3f}) [{label_names[corpus_labels[i]]}] {corpus_texts[i]}")
    rows_hybrid.append({
        "rank": k,
        "score": float(s),
        "doc_id": int(i),
        "label": label_names[corpus_labels[i]],
        "text": corpus_texts[i]
    })

- (0.158) [Business] Indians fill rail skills shortage Network Rail flies in specialist Indian engineers to work on the West Coast Mainline because of a UK skills shortage.
- (0.129) [Sports] QBs Grab Spotlight Chris Simms, the Buccaneers second-year pro competing for Tampa Bay's No. 2 job, completed 12 of 15 passes for 110 yards to help the Bucs to a 20-6 victory over the Cincinnati Bengals on Monday.
- (0.126) [World] Device 'may aid kidney dialysis' Scientists are developing a device which could improve the lives of kidney dialysis patients.
- (0.120) [Business] Not All Jobs Belong To The White Man: Asian Minorities, Affirmative Action, And The Quest For Parity At Work Although a smattering of Chinese, Filipinos, Japanese, Indians, Thais, and others may crow about seeing their kind sitting in prominent positions in corporations and organizations in the USA, these accomplishments become mere cultural high-fives and ritualistic chest-thumping goaded and impishly patronized by 'mainstr

In [139]:
# ------------------------
# CSV 저장 (각각)
# ------------------------
# - 출력 경로: ./outputs/
# - 인코딩: UTF-8 (엑셀에서 바로 열 때는 'utf-8-sig'도 고려)
out = os.path.join(os.path.abspath("."), "data")
os.makedirs(out, exist_ok=True)

pd.DataFrame(rows_tfidf).to_csv(os.path.join(out, "./nlp/tfidf_top5.csv"),
                                index=False, encoding="utf-8")
pd.DataFrame(rows_sbert).to_csv(os.path.join(out, "./nlp/sbert_top5.csv"),
                                index=False, encoding="utf-8")
pd.DataFrame(rows_hybrid).to_csv(os.path.join(out, "./nlp/hybrid_top5.csv"),
                                 index=False, encoding="utf-8")



### 4. 결과 비교

In [140]:
# ------------------------
# 비교용 테이블/프린트
# ------------------------

def show_results(title: str, pairs):
    """
    검색 결과 목록을 깔끔하게 프린트하는 유틸
    Args:
        title (str): 섹션 제목
        pairs (List[Tuple[int, float]]): (문서 인덱스, 점수) 리스트
    """
    print(f"\n[{title}]")
    for rank, (i, s) in enumerate(pairs, start=1):
        # 줄바꿈 제거 + 앞부분만 미리보기
        txt = corpus_texts[i].replace("\n", " ")
        print(f"{rank:>2}. ({s:.3f}) [{label_names[corpus_labels[i]]}] {txt}")

# 각 검색기에서 상위 5개 문서 인덱스/점수 가져오기
r_tfidf = search_tfidf(query, topk=5)
r_sbert = search_sbert(query, topk=5)
r_hybrid = search_hybrid(query, topk=5, alpha=0.6)  # SBERT 가중치 0.6

# 결과 프린트
show_results("TF-IDF Top-5", r_tfidf)
show_results("SBERT Top-5", r_sbert)
show_results("Hybrid(0.6) Top-5", r_hybrid)



[TF-IDF Top-5]
 1. (0.117) [Business] Will Schwab Reward Patience? The company saw an improvement in its trades, but will this market be kind to the brokerages?
 2. (0.116) [Business] Krispy Kreme COO Leaves for New Job  CHICAGO (Reuters) - Krispy Kreme Doughnuts Inc.  &lt;A HREF="http://www.investor.reuters.com/FullQuote.aspx?ticker=KKD.N target=/stocks/quickinfo/fullquote"&gt;KKD.N&lt;/A&gt;  said on Monday that its chief operating officer is leaving the  company for another job, the latest setback for a company that  is trying to recover from sluggish sales and an accounting  probe.
 3. (0.086) [Sports] DiMarco, Riley Get on Ryder Cup Team (AP) AP - Hal Sutton had a good idea what kind of U.S. team he would take to the Ryder Cup. All that changed in the final round of the PGA Championship.
 4. (0.085) [World] President's fate is on the line in Venezuela CARACAS -- Partisans on both sides are calling it the most polarized and important election in Venezuela's history, a presidential

In [141]:
# ------------------------
# 간단한 직관 비교: 레이블 빈도
# ------------------------
from collections import Counter

def label_counts(pairs):
    """
    (문서 인덱스, 점수) 리스트 → 레이블명 카운트 딕셔너리
    """
    return Counter([label_names[corpus_labels[i]] for i, _ in pairs])

print("\n[레이블 빈도 비교]")
for name, pairs in [("TF-IDF", r_tfidf), ("SBERT", r_sbert), ("Hybrid", r_hybrid)]:
    print(name, dict(label_counts(pairs)))



[레이블 빈도 비교]
TF-IDF {'Business': 2, 'Sports': 2, 'World': 1}
SBERT {'Business': 3, 'World': 1, 'Sci/Tech': 1}
Hybrid {'Business': 2, 'World': 1, 'Sci/Tech': 1, 'Sports': 1}


In [142]:
# ------------------------
# 문맥 번역 비교 : 번역 내용 비교
# ------------------------

In [143]:
import os                           # 환경변수 접근(os.getenv) 및 경로 검사(os.path.exists)용
from dotenv import load_dotenv      # .env 파일을 읽어 환경변수로 로드
from google.oauth2 import service_account  # 서비스 계정 기반 Credentials 생성을 위한 모듈
from google.cloud import translate_v2 as translate


In [144]:
# 번역 클라이언트 생성
def make_translate_client():
    cred_path = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
    if cred_path and os.path.exists(cred_path):
        creds = service_account.Credentials.from_service_account_file(cred_path)
        return translate.Client(credentials=creds)
    return translate.Client() 

In [145]:
import html

def translate_text(text: str, target_lang: str, source_lang: str | None = None) -> str:

    client = make_translate_client()
    resp = client.translate(text, target_language=target_lang, source_language=source_lang)
    # v2 API는 HTML-escaped 문자열을 반환하므로 unescape, 이상한 발음/기호 낭독 방지
    return html.unescape(resp["translatedText"])

In [146]:
# CSV 파일 경로
input_file = "./data/nlp/tfidf_top5.csv"
output_file = "./data/nlp/tfidf_top5_translated.csv"

# CSV 파일 읽기
df = pd.read_csv(input_file)

# text 컬럼 번역
df['translated_text'] = df['text'].apply(lambda x: translate_text(x, target_lang='ko'))

# 번역된 결과를 새로운 CSV 파일로 저장
df.to_csv(output_file, index=False, encoding='utf-8-sig')
tfidf_top5 = df.copy()

In [147]:
# CSV 파일 경로
input_file = "./data/nlp/sbert_top5.csv"
output_file = "./data/nlp/sbert_top5_translated.csv"

# CSV 파일 읽기
df = pd.read_csv(input_file)

# text 컬럼 번역
df['translated_text'] = df['text'].apply(lambda x: translate_text(x, target_lang='ko'))

# 번역된 결과를 새로운 CSV 파일로 저장
df.to_csv(output_file, index=False, encoding='utf-8-sig')
sbert_top5 = df.copy()

In [148]:
# CSV 파일 경로
input_file = "./data/nlp/hybrid_top5.csv"
output_file = "./data/nlp/hybrid_top5_translated.csv"

# CSV 파일 읽기
df = pd.read_csv(input_file)

# text 컬럼 번역
df['translated_text'] = df['text'].apply(lambda x: translate_text(x, target_lang='ko'))

# 번역된 결과를 새로운 CSV 파일로 저장
df.to_csv(output_file, index=False, encoding='utf-8-sig')
hybrid_top5 = df.copy()

In [152]:
tfidf_top5['translated_text'][1]


'크리스피 크림 최고운영책임자(COO)가 새로운 직장으로 떠나 시카고(로이터) - 크리스피 크림 도넛(Krispy Kreme Doughnuts Inc.) <A HREF="http://www.investor.reuters.com/FullQuote.aspx?ticker=KKD.N target=/stocks/quickinfo/fullquote">KKD.N</A>은 월요일에 최고운영책임자가 다른 직장으로 회사를 떠난다고 밝혔습니다. 이는 부진한 매출과 회계 조사에서 회복하려고 노력 중인 회사가 겪게 된 가장 큰 좌절입니다.'

In [153]:
sbert_top5['translated_text'][1]

"이 장치는 '신장 투석에 도움이 될 수 있다' 과학자들은 신장 투석 환자의 삶을 개선할 수 있는 장치를 개발하고 있습니다."

In [154]:
hybrid_top5['translated_text'][1]

'쿼터백, 주목을 받다 탬파베이의 2인자 자리를 놓고 경쟁하는 버카니어스 2년차 프로 선수 크리스 심스는 월요일에 신시내티 벵갈스를 상대로 15개의 패스 중 12개를 성공시켜 110야드를 달리며 버카니어스가 20대 6으로 승리하는 데 기여했습니다.'